### Часть 2. Генерация отчетов

#### 4. Генерация отчетов о движении

Набор полей:
1. flight number
2. movement type (тут только два варианта: arrival и departure)
3. date
4. time (UTC)

In [21]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('generated_flights.csv')
raw_data = pd.read_csv('raw_data.csv')
raw_data.head()

,Unnamed: 0,id,name,country,lat,lon
0,0,KVQQ,CECIL,United States of America,30.218778,-81.877167
1,1,KNQX,KEY WEST NAS/BOCA CHICA FIELD,United States of America,24.574636,-81.686644
2,2,KEYW,KEY WEST INTL,United States of America,24.556111,-81.759556
3,3,KISM,KISSIMMEE GATEWAY,United States of America,28.289806,-81.437083
4,4,KLAL,LAKELAND LINDER RGNL,United States of America,27.988917,-82.018556


In [2]:
flights = []
# Генерация дат для полетов только с января 2021 года по март 2021 года
flight_dates = pd.date_range(start="2021/01/01", end="2021/03/31", freq="D")

for row in df.iterrows():
    tmp = []
    schedule = row[1]["weekly_schedule"]
    flight_number = row[1]["flight_number"]
    off_time = row[1]["off_block_time"]
    on_time = row[1]["on_block_time"]
    
    # Генерация полетов исходя в соответствии с расписанием рейсов
    for dt in flight_dates:
        dt = pd.to_datetime(dt)
        if schedule[dt.weekday()] == "-":
            continue

        # Вылет
        tmp.append({
            "flight_number": flight_number,
            "movement_type": "departure",
            "date": dt.strftime("%d.%m.%Y"),
            "time": off_time,
        })

        # Прибытие
        days = 0
        if on_time < off_time:
            days = 1
        tmp.append({
            "flight_number": flight_number,
            "movement_type": "arrival",
            "date": (dt + timedelta(days=days)).strftime("%d.%m.%Y"),
            "time": on_time,
        })
    flights.extend(tmp)

In [3]:
# Создание датафрейма и разделение его на прибытие и вылет для удобства обработки 
all_flights = pd.DataFrame(flights)
all_flights.to_csv("generated_schedule.csv")

departures = all_flights.loc[all_flights["movement_type"] == "departure", :]
arrivals = all_flights.loc[all_flights["movement_type"] == "arrival", :]

all_flights.head()

,flight_number,movement_type,date,time
0,9876,departure,01.01.2021,23:00
1,9876,arrival,01.01.2021,23:48
2,9876,departure,02.01.2021,23:00
3,9876,arrival,02.01.2021,23:48
4,9876,departure,05.01.2021,23:00


In [4]:
# Сопоставляем вылет с прилетом
schedule = all_flights.loc[:, :]

schedule["arrival_time"] = schedule["time"].shift(-1)
schedule["arrival_date"] = schedule["date"].shift(-1)
schedule = schedule.loc[schedule[schedule["movement_type"] == "departure"].index, :]
schedule.columns = [
    "flight_number", "movement_type", "departure_date", "departure_time", "arrival_time", "arrival_date"
]
schedule = schedule.reset_index(drop=True)
schedule.head()

,flight_number,movement_type,departure_date,departure_time,arrival_time,arrival_date
0,9876,departure,01.01.2021,23:00,23:48,01.01.2021
1,9876,departure,02.01.2021,23:00,23:48,02.01.2021
2,9876,departure,05.01.2021,23:00,23:48,05.01.2021
3,9876,departure,06.01.2021,23:00,23:48,06.01.2021
4,9876,departure,07.01.2021,23:00,23:48,07.01.2021


In [5]:
# Конвертируем столбцы с датой и временем в datetime 
schedule["departure_datetime"] = schedule.apply(lambda x: x["departure_date"] + " " + x["departure_time"], axis=1)
schedule["arrival_datetime"] = schedule.apply(lambda x: x["arrival_date"] + " " + x["arrival_time"], axis=1)
 
schedule["departure_datetime"] = pd.to_datetime(schedule["departure_datetime"], format="%d.%m.%Y %H:%M")
schedule["arrival_datetime"] = pd.to_datetime(schedule["arrival_datetime"], format="%d.%m.%Y %H:%M")

schedule.head()

,flight_number,movement_type,departure_date,departure_time,arrival_time,arrival_date,departure_datetime,arrival_datetime
0,9876,departure,01.01.2021,23:00,23:48,01.01.2021,2021-01-01 23:00:00,2021-01-01 23:48:00
1,9876,departure,02.01.2021,23:00,23:48,02.01.2021,2021-01-02 23:00:00,2021-01-02 23:48:00
2,9876,departure,05.01.2021,23:00,23:48,05.01.2021,2021-01-05 23:00:00,2021-01-05 23:48:00
3,9876,departure,06.01.2021,23:00,23:48,06.01.2021,2021-01-06 23:00:00,2021-01-06 23:48:00
4,9876,departure,07.01.2021,23:00,23:48,07.01.2021,2021-01-07 23:00:00,2021-01-07 23:48:00


In [6]:
# Летаем 95% рейсов (т.е. какие-то не летаем совсем)

# Для каждой выборки алгоритм один и тот же - найти размер определенного процента (из ТЗ), 
# взять рандомную выборку по кол-ву этого размера и сохранить их индексы для дальнейшей обработки и для того, 
# чтобы выборки не пересекались

perc5 = int(5 * schedule.shape[0] / 100)
canceled_ids = set(schedule.sample(perc5).index.tolist())
schedule.loc[:, "status"] = schedule.index.map(lambda x: "canceled" if x in canceled_ids else "")
schedule.head()

,flight_number,movement_type,departure_date,departure_time,arrival_time,arrival_date,departure_datetime,arrival_datetime,status
0,9876,departure,01.01.2021,23:00,23:48,01.01.2021,2021-01-01 23:00:00,2021-01-01 23:48:00,
1,9876,departure,02.01.2021,23:00,23:48,02.01.2021,2021-01-02 23:00:00,2021-01-02 23:48:00,
2,9876,departure,05.01.2021,23:00,23:48,05.01.2021,2021-01-05 23:00:00,2021-01-05 23:48:00,
3,9876,departure,06.01.2021,23:00,23:48,06.01.2021,2021-01-06 23:00:00,2021-01-06 23:48:00,
4,9876,departure,07.01.2021,23:00,23:48,07.01.2021,2021-01-07 23:00:00,2021-01-07 23:48:00,


In [7]:
# 20% рейсов опаздывают (рандомно от 15-и минут до часа)
perc20_arrival = int(20*schedule[schedule['status'] == ''].shape[0] / 100)
delayed_ids = schedule[schedule['status'] == ''].sample(perc20_arrival).index.tolist()
schedule.iloc[delayed_ids, -1] = "delayed"
schedule.head()

,flight_number,movement_type,departure_date,departure_time,arrival_time,arrival_date,departure_datetime,arrival_datetime,status
0,9876,departure,01.01.2021,23:00,23:48,01.01.2021,2021-01-01 23:00:00,2021-01-01 23:48:00,delayed
1,9876,departure,02.01.2021,23:00,23:48,02.01.2021,2021-01-02 23:00:00,2021-01-02 23:48:00,delayed
2,9876,departure,05.01.2021,23:00,23:48,05.01.2021,2021-01-05 23:00:00,2021-01-05 23:48:00,
3,9876,departure,06.01.2021,23:00,23:48,06.01.2021,2021-01-06 23:00:00,2021-01-06 23:48:00,
4,9876,departure,07.01.2021,23:00,23:48,07.01.2021,2021-01-07 23:00:00,2021-01-07 23:48:00,


In [8]:
# 50% опаздавших, опаздывают из-за позднего вылета, остальные вылетают во время, но опаздывают по прибытии (сильный встречный ветер)
from random import randint

perc50_delayed1 = int(50*schedule[schedule['status'] == 'delayed'].shape[0] / 100)
late_departed_ids = set(schedule[schedule['status'] == 'delayed'].sample(perc50_delayed1).index.tolist())

late_dearture_ids_revert = set(
    schedule[(schedule['status'] == 'delayed') & (~schedule.index.isin(late_departed_ids))].index.tolist()
)

def get_rnd_shift_dep(row):
    if row.name in late_departed_ids:
        shift = timedelta(minutes=randint(15, 60))
        row["departure_datetime"] = row["departure_datetime"] + shift
        row["arrival_datetime"] = row["arrival_datetime"] + shift
    return row

def get_rnd_shift_arr(row):
    if row.name in late_dearture_ids_revert:
        return row["arrival_datetime"] + timedelta(minutes=randint(15, 60))
    return row["arrival_datetime"]

schedule = schedule.apply(get_rnd_shift_dep, axis=1)
schedule["arrival_datetime"] = schedule.apply(get_rnd_shift_arr, axis=1)
schedule.head()

,flight_number,movement_type,departure_date,departure_time,arrival_time,arrival_date,departure_datetime,arrival_datetime,status
0,9876,departure,01.01.2021,23:00,23:48,01.01.2021,2021-01-01 23:50:00,2021-01-02 00:38:00,delayed
1,9876,departure,02.01.2021,23:00,23:48,02.01.2021,2021-01-02 23:00:00,2021-01-03 00:03:00,delayed
2,9876,departure,05.01.2021,23:00,23:48,05.01.2021,2021-01-05 23:00:00,2021-01-05 23:48:00,
3,9876,departure,06.01.2021,23:00,23:48,06.01.2021,2021-01-06 23:00:00,2021-01-06 23:48:00,
4,9876,departure,07.01.2021,23:00,23:48,07.01.2021,2021-01-07 23:00:00,2021-01-07 23:48:00,


In [9]:
# 30% выполненных рейсов вылетают позже расписания на 17-23 минуты (рандомно), но прилетают по время
perc30_ontime = int(30*schedule[schedule['status'] == '' ].shape[0] / 100)
perc30_ontime_ids = set(schedule[schedule['status'] == '' ].sample(perc30_ontime).index.tolist())

def get_rnd_shift_dep(row):
    if row.name in perc30_ontime_ids:
        return row["departure_datetime"] + timedelta(minutes=randint(17, 23))
    return row["departure_datetime"]

schedule["departure_datetime"] = schedule.apply(get_rnd_shift_dep, axis=1)
schedule.head()

,flight_number,movement_type,departure_date,departure_time,arrival_time,arrival_date,departure_datetime,arrival_datetime,status
0,9876,departure,01.01.2021,23:00,23:48,01.01.2021,2021-01-01 23:50:00,2021-01-02 00:38:00,delayed
1,9876,departure,02.01.2021,23:00,23:48,02.01.2021,2021-01-02 23:00:00,2021-01-03 00:03:00,delayed
2,9876,departure,05.01.2021,23:00,23:48,05.01.2021,2021-01-05 23:00:00,2021-01-05 23:48:00,
3,9876,departure,06.01.2021,23:00,23:48,06.01.2021,2021-01-06 23:23:00,2021-01-06 23:48:00,
4,9876,departure,07.01.2021,23:00,23:48,07.01.2021,2021-01-07 23:00:00,2021-01-07 23:48:00,


In [10]:
# 5% выполненных рейсов вылетают на 17-23 (рандомно) минут раньше и прилетают на 17-23 (рандомно) минут раньше
slice1 = schedule[(schedule['status'] == '')]
slice2 = slice1[~slice1.isin(perc30_ontime_ids)]
perc5_ontime = int(5*slice2.shape[0] / 100)
perc5_ontime_ids = set(slice2.sample(perc5_ontime).index.tolist())

def get_rnd_shift_dep(row):
    if row.name in perc5_ontime_ids:
        shift = timedelta(minutes=randint(17, 23))
        row["departure_datetime"] = row["departure_datetime"] - shift
        row["arrival_datetime"] = row["arrival_datetime"] - shift
    return row

schedule = schedule.apply(get_rnd_shift_dep, axis=1)
schedule.head()

,flight_number,movement_type,departure_date,departure_time,arrival_time,arrival_date,departure_datetime,arrival_datetime,status
0,9876,departure,01.01.2021,23:00,23:48,01.01.2021,2021-01-01 23:50:00,2021-01-02 00:38:00,delayed
1,9876,departure,02.01.2021,23:00,23:48,02.01.2021,2021-01-02 23:00:00,2021-01-03 00:03:00,delayed
2,9876,departure,05.01.2021,23:00,23:48,05.01.2021,2021-01-05 23:00:00,2021-01-05 23:48:00,
3,9876,departure,06.01.2021,23:00,23:48,06.01.2021,2021-01-06 23:23:00,2021-01-06 23:48:00,
4,9876,departure,07.01.2021,23:00,23:48,07.01.2021,2021-01-07 23:00:00,2021-01-07 23:48:00,


In [11]:
# Приводим таблицу вылетов к виду, указанному в ТЗ (4 столбца)
arrivals = schedule.loc[:, :]
arrivals["movement_type"] = "arrival"
arrivals = arrivals.drop(
    columns=["departure_date", "departure_time", "arrival_time", "arrival_date", "departure_datetime", "status"]
)
arrivals.columns = ["flight_number", "movement_type", "datetime"]
arrivals.head()

,flight_number,movement_type,datetime
0,9876,arrival,2021-01-02 00:38:00
1,9876,arrival,2021-01-03 00:03:00
2,9876,arrival,2021-01-05 23:48:00
3,9876,arrival,2021-01-06 23:48:00
4,9876,arrival,2021-01-07 23:48:00


In [12]:
# Приводим таблицу прилетов к виду, указанному в ТЗ (4 столбца)
departures = schedule.loc[:, :]
departures["movement_type"] = "departure"
departures = departures.drop(
    columns=["departure_date", "departure_time", "arrival_time", "arrival_date", "arrival_datetime", "status"]
)
departures.columns = ["flight_number", "movement_type", "datetime"]
departures.head()

,flight_number,movement_type,datetime
0,9876,departure,2021-01-01 23:50:00
1,9876,departure,2021-01-02 23:00:00
2,9876,departure,2021-01-05 23:00:00
3,9876,departure,2021-01-06 23:23:00
4,9876,departure,2021-01-07 23:00:00


In [13]:
# Соединяем 2 таблицы
traffic_report = pd.concat([arrivals, departures])
traffic_report.head()

,flight_number,movement_type,datetime
0,9876,arrival,2021-01-02 00:38:00
1,9876,arrival,2021-01-03 00:03:00
2,9876,arrival,2021-01-05 23:48:00
3,9876,arrival,2021-01-06 23:48:00
4,9876,arrival,2021-01-07 23:48:00


In [14]:
# Обрабатываем данные по ТЗ перед сохранением
traffic_report["date"] = traffic_report["datetime"].dt.strftime("%d.%m.%Y")
traffic_report["time"] = traffic_report["datetime"].dt.strftime("%H:%M")
traffic_report = traffic_report.drop(columns=["datetime"])
traffic_report = traffic_report.sort_values(by="date")
traffic_report = traffic_report.reset_index(drop=True)
traffic_report.head()

,flight_number,movement_type,date,time
0,4058,departure,01.01.2021,19:00
1,8230,departure,01.01.2021,23:00
2,2164,departure,01.01.2021,20:30
3,5478,departure,01.01.2021,17:00
4,7403,departure,01.01.2021,22:30


In [15]:
# Сохраняем полученный датасет
traffic_report.to_csv("traffic_report.csv")

### Часть 3. Импорт сгенерированных данных в базу (PostgreSQL)

In [22]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# установка postgresql на macos через brew,
# запуск brew services start postgresql
# Поместить свои данные
conn_string = 'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'.format(
    user="",
    password="",
    host="localhost",
    port="5432",
    dbname="postgres",
)
  
db = create_engine(conn_string)
conn = db.connect()
traffic_report.to_sql('traffic_report', con=conn, if_exists='replace', index=False)
df.to_sql('schedule', con=conn, if_exists='replace', index=False)
raw_data.to_sql('ref_airports', con=conn, if_exists='replace', index=False)

conn.close()